In [18]:
import pandas as pd
import os
import re
import numpy as np
import sys 
import config_options as cfg
chosen_day = cfg.chosen_day
print(cfg.carpeta_md_opciones)
print(chosen_day)

C:\Users\Pedro\Research\Options\market_data\Opciones
2025-05-05


In [17]:
def cargar_csv(carpeta, chosen_day):
    files = [f for f in os.listdir(carpeta) if chosen_day in f and f.endswith(".csv")]
    if files:
        full_path = os.path.join(carpeta, files[0])
        return pd.read_csv(full_path)
    else:
        print(f"Dia {chosen_day} no encontrado en {carpeta}. Revisa en la carpeta")
        return None

data_base = cargar_csv(cfg.carpeta_md_opciones, chosen_day)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\Pedro\\Research\\Options\\market_data\\Opciones'